# Beach Crowd Prediction — LSTM, TFT, TiDE + Optuna Optimization

This notebook compares models across **three dataset strategies**:
1. **Daytime only** — remove night hours (6PM–6AM)
2. **Full 24h** — keep all data including noisy night counts
3. **Night = 0** — keep 24h but replace night counts with 0

Models tested:
- **Sklearn baselines**: Lasso, RandomForest, XGBoost, LightGBM, CatBoost
- **NeuralForecast**: LSTM, TFT, TiDE
- **Optuna-optimized**: XGBoost, LightGBM, CatBoost, LSTM

In [ ]:
# === PATHS ===
CACHE_DIR = "cache/predictions"
COUNTING_MODEL = "bayesian_vgg19"
SAVE_DIR = "models/optuna_comparison"

# === SAMPLING ===
SAMPLE_FRAC = 1.0
MAX_BEACHES = None

# === MODEL PARAMETERS ===
MAX_STEPS = 500
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
INPUT_SIZE = 24

# === TIME ===
NIGHT_START = 20
NIGHT_END = 6

# === OPTUNA ===
OPTUNA_TRIALS = 30
OPTUNA_TIMEOUT = 300

# === FLAGS ===
RUN_SKLEARN = True
RUN_NEURALFORECAST = True
RUN_OPTUNA = True

In [ ]:
import subprocess, sys
pkgs = ["neuralforecast", "xgboost", "lightgbm", "catboost", "utilsforecast", "optuna"]
for pkg in pkgs:
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])
print("Packages installed")

In [ ]:
import json, time, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso
import torch

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except: HAS_XGB = False

try:
    from lightgbm import LGBMRegressor
    HAS_LGBM = True
except: HAS_LGBM = False

try:
    from catboost import CatBoostRegressor
    HAS_CATBOOST = True
except: HAS_CATBOOST = False

try:
    from neuralforecast import NeuralForecast
    from neuralforecast.models import LSTM, TFT, TiDE
    from neuralforecast.losses.pytorch import MAE
    HAS_NF = True
except Exception as e:
    print(f"NeuralForecast error: {e}")
    HAS_NF = False

if torch.cuda.is_available():
    ACCELERATOR = 'gpu'
    DEVICES = 1
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    ACCELERATOR = 'mps'
    DEVICES = 1
else:
    ACCELERATOR = 'cpu'
    DEVICES = 1

print(f"Accelerator: {ACCELERATOR}")
print(f"XGB: {HAS_XGB}, LGBM: {HAS_LGBM}, CatBoost: {HAS_CATBOOST}, NF: {HAS_NF}")

In [ ]:
def calc_metrics(y_true, y_pred, max_count):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    rel_mae = (mae / max_count) * 100 if max_count > 0 else 0
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'RelMAE': rel_mae}

def eval_per_beach(df, y_pred, beach_col='unique_id'):
    results = []
    for b in df[beach_col].unique():
        mask = df[beach_col] == b
        if mask.sum() < 3:
            continue
        y_true = df.loc[mask, 'y'].values if 'y' in df.columns else df.loc[mask, 'count'].values
        y_p = y_pred[mask.values] if hasattr(mask, 'values') else y_pred[mask]
        max_count = y_true.max()
        m = calc_metrics(y_true, y_p, max_count)
        m['camera'] = b
        m['max_count'] = max_count
        m['n'] = mask.sum()
        results.append(m)
    return pd.DataFrame(results)

## Load and Prepare Data

In [ ]:
def load_cache(cache_dir, model):
    cache_path = Path(cache_dir) / model
    records = []
    for jf in cache_path.rglob("*.json"):
        try:
            with open(jf) as f:
                r = json.load(f)
            if 'error' not in r:
                records.append(r)
        except: pass
    
    rows = []
    for r in records:
        row = {
            'beach': r.get('beach') or r.get('beach_folder'),
            'beach_folder': r.get('beach_folder'),
            'datetime': r.get('datetime'),
            'count': r.get('count')
        }
        for k, v in r.get('weather', {}).items():
            row[k] = v
        rows.append(row)
    
    df = pd.DataFrame(rows)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values('datetime').reset_index(drop=True)
    return df

df_raw = load_cache(CACHE_DIR, COUNTING_MODEL)
print(f"Loaded: {len(df_raw)} rows, {df_raw['beach'].nunique()} beaches")

In [ ]:
EXCLUDE = ['livecampro/001', 'livecampro/011', 'livecampro/018', 'livecampro/021',
    'livecampro/030', 'livecampro/039', 'livecampro/070', 'MultimediaTres/PortAndratx',
    'SeeTheWorld/mallorca_pancam', 'skyline/es-pujols']
EXCLUDE_PREFIX = ['ibred', 'ClubNauticSoller', 'Guenthoer', 'youtube']

before = len(df_raw)
df_raw = df_raw[~df_raw['beach_folder'].isin(EXCLUDE)]
for p in EXCLUDE_PREFIX:
    df_raw = df_raw[~df_raw['beach_folder'].str.startswith(p, na=False)]
print(f"Filtered: {before} -> {len(df_raw)}")

In [ ]:
if SAMPLE_FRAC < 1.0:
    df_raw = df_raw.sample(frac=SAMPLE_FRAC, random_state=42).sort_values('datetime').reset_index(drop=True)

if MAX_BEACHES:
    top = df_raw['beach'].value_counts().head(MAX_BEACHES).index.tolist()
    df_raw = df_raw[df_raw['beach'].isin(top)].reset_index(drop=True)

print(f"Final: {len(df_raw)} rows, {df_raw['beach'].nunique()} beaches")

In [ ]:
df = df_raw.copy()
df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.dayofweek
df['month'] = df['datetime'].dt.month
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_summer'] = df['month'].isin([6, 7, 8]).astype(int)
df['is_night'] = ((df['hour'] >= NIGHT_START) | (df['hour'] <= NIGHT_END)).astype(int)

WEATHER_COLS = [c for c in df.columns if c.startswith('ae_') or c.startswith('om_')]
TEMPORAL_COLS = ['hour', 'day_of_week', 'month', 'is_weekend', 'is_summer', 'is_night']
ALL_FEATURES = WEATHER_COLS + TEMPORAL_COLS

df = df.dropna(subset=ALL_FEATURES + ['count']).reset_index(drop=True)
good = df.groupby('beach')['count'].max()
good = good[good > 20].index.tolist()
df = df[df['beach'].isin(good)].reset_index(drop=True)

print(f"After cleaning: {len(df)} rows, {len(good)} beaches")
print(f"Features: {len(ALL_FEATURES)}")

## Create Three Dataset Strategies

In [ ]:
ds_daytime = df[df['is_night'] == 0].copy().reset_index(drop=True)
ds_full24h = df.copy()
ds_night0 = df.copy()
ds_night0.loc[ds_night0['is_night'] == 1, 'count'] = 0.0

datasets = {'Daytime': ds_daytime, 'Full24h': ds_full24h, 'Night0': ds_night0}

print("=" * 80)
print("DATASET COMPARISON")
print("=" * 80)

for name, d in datasets.items():
    night_rows = d[d['is_night'] == 1] if 'is_night' in d.columns else pd.DataFrame()
    day_rows = d[d['is_night'] == 0] if 'is_night' in d.columns else d
    
    print(f"\n{name}:")
    print(f"  Total rows:     {len(d)}")
    print(f"  Beaches:        {d['beach'].nunique()}")
    print(f"  Night rows:     {len(night_rows)} ({len(night_rows)/len(d)*100:.1f}%)")
    print(f"  Day rows:       {len(day_rows)} ({len(day_rows)/len(d)*100:.1f}%)")
    print(f"  Count mean:     {d['count'].mean():.1f}")
    print(f"  Count max:      {d['count'].max():.1f}")
    print(f"  Zeros:          {(d['count'] == 0).sum()} ({(d['count'] == 0).sum()/len(d)*100:.1f}%)")

In [ ]:
def split_data(df, train_frac=0.7, val_frac=0.15):
    n = len(df)
    t1 = int(n * train_frac)
    t2 = int(n * (train_frac + val_frac))
    return df.iloc[:t1], df.iloc[t1:t2], df.iloc[t2:]

splits = {}
for name, d in datasets.items():
    train, val, test = split_data(d)
    splits[name] = {'train': train, 'val': val, 'test': test}
    print(f"{name}: train={len(train)}, val={len(val)}, test={len(test)}")

## Prepare Datasets with Gap Filling and Interpolation

In [ ]:
from utilsforecast.preprocessing import fill_gaps

def to_nf_format(df, id_col='beach_folder'):
    cols = ['datetime', id_col, 'count'] + ALL_FEATURES
    cols = [c for c in cols if c in df.columns]
    nf_df = df[cols].copy()
    nf_df = nf_df.rename(columns={'datetime': 'ds', id_col: 'unique_id', 'count': 'y'})
    return nf_df

def prepare_dataset_with_filled_gaps(train_df, test_df, freq='h'):
    nf_train = to_nf_format(train_df)
    nf_test = to_nf_format(test_df)
    
    nf_train = nf_train.groupby(['unique_id', 'ds']).mean(numeric_only=True).reset_index()
    nf_test = nf_test.groupby(['unique_id', 'ds']).mean(numeric_only=True).reset_index()
    
    nf_train = fill_gaps(nf_train, freq=freq)
    nf_test = fill_gaps(nf_test, freq=freq)
    
    numeric_cols = nf_train.select_dtypes(include=[np.number]).columns.tolist()
    for col in numeric_cols:
        nf_train[col] = nf_train.groupby('unique_id')[col].transform(
            lambda x: x.interpolate(method='linear').ffill().bfill()
        )
        nf_test[col] = nf_test.groupby('unique_id')[col].transform(
            lambda x: x.interpolate(method='linear').ffill().bfill()
        )
    
    common_ids = set(nf_train['unique_id'].unique()) & set(nf_test['unique_id'].unique())
    nf_train = nf_train[nf_train['unique_id'].isin(common_ids)].reset_index(drop=True)
    nf_test = nf_test[nf_test['unique_id'].isin(common_ids)].reset_index(drop=True)
    
    return nf_train, nf_test, list(common_ids)

prepared_data = {}

print("=" * 70)
print("PREPARING DATASETS WITH FILLED GAPS + INTERPOLATION")
print("=" * 70)

for ds_name in ['Daytime', 'Full24h', 'Night0']:
    print(f"\nProcessing: {ds_name}")
    s = splits[ds_name]
    train_val = pd.concat([s['train'], s['val']])
    
    nf_train, nf_test, series_ids = prepare_dataset_with_filled_gaps(train_val, s['test'])
    
    prepared_data[ds_name] = {
        'train': nf_train,
        'test': nf_test,
        'series_ids': series_ids,
        'n_series': len(series_ids),
    }
    
    print(f"  Train: {len(nf_train)}, Test: {len(nf_test)}, Series: {len(series_ids)}")
    print(f"  NaN check - train: {nf_train['y'].isna().sum()}, test: {nf_test['y'].isna().sum()}")

## Sklearn Baseline Models (across all datasets)

In [ ]:
all_results = []
all_beach_results = []

if RUN_SKLEARN:
    for ds_name in datasets.keys():
        data = prepared_data[ds_name]
        train_df = data['train']
        test_df = data['test']
        
        feature_cols = [c for c in train_df.columns if c not in ['unique_id', 'ds', 'y']]
        
        X_train = train_df[feature_cols]
        y_train = train_df['y']
        X_test = test_df[feature_cols]
        y_test = test_df['y']
        
        sklearn_models = {
            'Lasso': Lasso(alpha=0.1),
            'RandomForest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
        }
        if HAS_XGB:
            sklearn_models['XGBoost'] = XGBRegressor(n_estimators=200, max_depth=6, random_state=42, n_jobs=-1, verbosity=0)
        if HAS_LGBM:
            sklearn_models['LightGBM'] = LGBMRegressor(n_estimators=200, max_depth=6, random_state=42, n_jobs=-1, verbose=-1)
        if HAS_CATBOOST:
            sklearn_models['CatBoost'] = CatBoostRegressor(n_estimators=200, max_depth=6, random_state=42, verbose=0)
        
        print(f"\n{'=' * 60}")
        print(f"SKLEARN - {ds_name}")
        print(f"{'=' * 60}")
        print(f"Train: {len(X_train)}, Test: {len(X_test)}, Features: {len(feature_cols)}")
        
        for name, model in sklearn_models.items():
            t0 = time.time()
            model.fit(X_train, y_train)
            y_pred = np.clip(model.predict(X_test), 0, None)
            elapsed = time.time() - t0
            
            m = calc_metrics(y_test.values, y_pred, y_test.max())
            
            eval_df = test_df[['unique_id', 'y']].copy()
            eval_df['count'] = eval_df['y']
            eval_df['beach'] = eval_df['unique_id']
            beach_df = eval_per_beach(eval_df, y_pred, 'beach')
            beach_df['model'] = name
            beach_df['dataset'] = ds_name
            all_beach_results.append(beach_df)
            
            avg_rel = beach_df['RelMAE'].mean()
            all_results.append({
                'Model': name, 'Dataset': ds_name, 'Type': 'Sklearn',
                'MAE': m['MAE'], 'RMSE': m['RMSE'], 'R2': m['R2'],
                'AvgRelMAE': avg_rel, 'Time': elapsed
            })
            print(f"  {name:15s} | {elapsed:5.1f}s | MAE={m['MAE']:.1f} | RelMAE={avg_rel:.1f}% | R2={m['R2']:.3f}")

## NeuralForecast Models: LSTM, TFT, TiDE

| Model | Strengths | Best For |
|-------|-----------|----------|
| **LSTM** | Classic baseline, proven | Sequential patterns |
| **TFT** | Static features, interpretable attention | New beach prediction |
| **TiDE** | Fast, lightweight, exogenous support | Production speed |

In [ ]:
def get_nf_models(hist_exog, horizon):
    common = dict(
        h=horizon,
        input_size=INPUT_SIZE,
        max_steps=MAX_STEPS,
        early_stop_patience_steps=-1,
        learning_rate=LEARNING_RATE,
        batch_size=BATCH_SIZE,
        scaler_type='robust',
        random_seed=42,
        accelerator=ACCELERATOR,
        devices=DEVICES,
        loss=MAE(),
    )
    
    return [
        ('LSTM', LSTM(
            hist_exog_list=hist_exog,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            **common
        )),
        ('TFT', TFT(
            hist_exog_list=hist_exog,
            hidden_size=64,
            n_head=4,
            **common
        )),
        ('TiDE', TiDE(
            hist_exog_list=hist_exog,
            hidden_size=128,
            decoder_output_dim=16,
            **common
        )),
    ]

print("NF Models: LSTM, TFT, TiDE")

In [ ]:
if RUN_NEURALFORECAST and HAS_NF:
    for ds_name in ['Full24h', 'Night0']:
        nf_train = prepared_data[ds_name]['train']
        nf_test = prepared_data[ds_name]['test']
        
        nf_all = pd.concat([nf_train, nf_test]).sort_values(['unique_id', 'ds']).reset_index(drop=True)
        test_horizon = nf_test.groupby('unique_id').size().min()
        horizon = min(72, test_horizon)
        
        print(f"\n{'=' * 60}")
        print(f"NEURALFORECAST - {ds_name} (horizon={horizon})")
        print(f"{'=' * 60}")
        
        for model_name, model in get_nf_models(ALL_FEATURES, horizon=horizon):
            print(f"\n  {model_name}...")
            try:
                t0 = time.time()
                nf = NeuralForecast(models=[model], freq='h')
                
                cv_results = nf.cross_validation(
                    df=nf_all,
                    n_windows=1,
                    step_size=horizon,
                )
                elapsed = time.time() - t0
                
                pred_col = [c for c in cv_results.columns if c not in ['unique_id', 'ds', 'cutoff', 'y']][0]
                
                y_true = cv_results['y'].values
                y_pred = np.clip(cv_results[pred_col].values, 0, None)
                
                m = calc_metrics(y_true, y_pred, y_true.max())
                
                eval_df = cv_results.copy()
                eval_df['beach'] = eval_df['unique_id']
                beach_df = eval_per_beach(eval_df, y_pred, 'beach')
                avg_rel = beach_df['RelMAE'].mean() if len(beach_df) > 0 else np.nan
                
                all_results.append({
                    'Model': model_name, 'Dataset': ds_name, 'Type': 'NeuralForecast',
                    'MAE': m['MAE'], 'RMSE': m['RMSE'], 'R2': m['R2'],
                    'AvgRelMAE': avg_rel, 'Time': elapsed
                })
                print(f"    {elapsed:.1f}s | MAE={m['MAE']:.1f} | RelMAE={avg_rel:.1f}% | R2={m['R2']:.3f}")
                
            except Exception as e:
                print(f"    ERROR: {e}")
                import traceback
                traceback.print_exc()

## Optuna Hyperparameter Optimization

Optimize across datasets:
- **XGBoost**: n_estimators, max_depth, learning_rate, subsample, colsample_bytree
- **LightGBM**: n_estimators, num_leaves, learning_rate, feature_fraction, bagging_fraction
- **CatBoost**: iterations, depth, learning_rate, l2_leaf_reg
- **LSTM** (via NeuralForecast): hidden_size, n_layers, learning_rate, dropout

In [ ]:
def calc_avg_rel_mae(y_true, y_pred, groups):
    rel_maes = []
    for g in np.unique(groups):
        mask = groups == g
        if mask.sum() < 3:
            continue
        yt = y_true[mask]
        yp = y_pred[mask]
        max_count = yt.max()
        if max_count > 0:
            rel_maes.append(mean_absolute_error(yt, yp) / max_count * 100)
    return np.mean(rel_maes) if rel_maes else 999.0

def create_xgb_objective(X_train, y_train, X_val, y_val, groups_val):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
            'random_state': 42, 'n_jobs': -1, 'verbosity': 0,
        }
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = np.clip(model.predict(X_val), 0, None)
        return calc_avg_rel_mae(y_val.values, y_pred, groups_val.values)
    return objective

def create_lgbm_objective(X_train, y_train, X_val, y_val, groups_val):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'num_leaves': trial.suggest_int('num_leaves', 20, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
            'random_state': 42, 'n_jobs': -1, 'verbose': -1,
        }
        model = LGBMRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = np.clip(model.predict(X_val), 0, None)
        return calc_avg_rel_mae(y_val.values, y_pred, groups_val.values)
    return objective

def create_catboost_objective(X_train, y_train, X_val, y_val, groups_val):
    def objective(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 100, 1000),
            'depth': trial.suggest_int('depth', 4, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
            'random_strength': trial.suggest_float('random_strength', 1e-8, 10.0, log=True),
            'border_count': trial.suggest_int('border_count', 32, 255),
            'random_state': 42, 'verbose': 0,
        }
        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = np.clip(model.predict(X_val), 0, None)
        return calc_avg_rel_mae(y_val.values, y_pred, groups_val.values)
    return objective

In [ ]:
best_models_per_dataset = {}

if RUN_OPTUNA:
    for ds_name in ['Night0']:
        data = prepared_data[ds_name]
        
        feature_cols = [c for c in data['train'].columns if c not in ['unique_id', 'ds', 'y']]
        
        train_nf = data['train']
        
        split_idx = int(len(train_nf) * 0.82)
        X_tr = train_nf[feature_cols].iloc[:split_idx]
        y_tr = train_nf['y'].iloc[:split_idx]
        X_va = train_nf[feature_cols].iloc[split_idx:]
        y_va = train_nf['y'].iloc[split_idx:]
        groups_va = train_nf['unique_id'].iloc[split_idx:]
        
        best_models = {}
        
        print(f"\n{'=' * 60}")
        print(f"OPTUNA - {ds_name} (optimizing AvgRelMAE, trials={OPTUNA_TRIALS})")
        print(f"{'=' * 60}")
        
        if HAS_XGB:
            print("\n  XGBoost...")
            study = optuna.create_study(direction='minimize')
            study.optimize(create_xgb_objective(X_tr, y_tr, X_va, y_va, groups_va),
                          n_trials=OPTUNA_TRIALS, timeout=OPTUNA_TIMEOUT, show_progress_bar=True)
            best_models['XGBoost_Optuna'] = study.best_params
            print(f"    Best AvgRelMAE: {study.best_value:.2f}%")
        
        if HAS_LGBM:
            print("\n  LightGBM...")
            study = optuna.create_study(direction='minimize')
            study.optimize(create_lgbm_objective(X_tr, y_tr, X_va, y_va, groups_va),
                          n_trials=OPTUNA_TRIALS, timeout=OPTUNA_TIMEOUT, show_progress_bar=True)
            best_models['LightGBM_Optuna'] = study.best_params
            print(f"    Best AvgRelMAE: {study.best_value:.2f}%")
        
        if HAS_CATBOOST:
            print("\n  CatBoost...")
            study = optuna.create_study(direction='minimize')
            study.optimize(create_catboost_objective(X_tr, y_tr, X_va, y_va, groups_va),
                          n_trials=OPTUNA_TRIALS, timeout=OPTUNA_TIMEOUT, show_progress_bar=True)
            best_models['CatBoost_Optuna'] = study.best_params
            print(f"    Best AvgRelMAE: {study.best_value:.2f}%")
        
        best_models_per_dataset[ds_name] = best_models

In [ ]:
if RUN_OPTUNA and best_models_per_dataset:
    print("\n" + "=" * 60)
    print("EVALUATING OPTUNA MODELS ACROSS ALL DATASETS")
    print("=" * 60)
    
    for ds_name in datasets.keys():
        data = prepared_data[ds_name]
        feature_cols = [c for c in data['train'].columns if c not in ['unique_id', 'ds', 'y']]
        
        X_train_full = data['train'][feature_cols]
        y_train_full = data['train']['y']
        X_test = data['test'][feature_cols]
        y_test = data['test']['y']
        
        optuna_ds = 'Night0'
        best_models = best_models_per_dataset.get(optuna_ds, {})
        
        print(f"\n  --- {ds_name} ---")
        
        for name, params in best_models.items():
            t0 = time.time()
            
            if 'XGBoost' in name:
                model = XGBRegressor(**params, random_state=42, n_jobs=-1, verbosity=0)
            elif 'LightGBM' in name:
                model = LGBMRegressor(**params, random_state=42, n_jobs=-1, verbose=-1)
            elif 'CatBoost' in name:
                model = CatBoostRegressor(**params, random_state=42, verbose=0)
            else:
                continue
            
            model.fit(X_train_full, y_train_full)
            y_pred = np.clip(model.predict(X_test), 0, None)
            elapsed = time.time() - t0
            
            m = calc_metrics(y_test.values, y_pred, y_test.max())
            
            eval_df = data['test'][['unique_id', 'y']].copy()
            eval_df['count'] = eval_df['y']
            eval_df['beach'] = eval_df['unique_id']
            beach_df = eval_per_beach(eval_df, y_pred, 'beach')
            avg_rel = beach_df['RelMAE'].mean()
            
            all_results.append({
                'Model': name, 'Dataset': ds_name, 'Type': 'Optuna',
                'MAE': m['MAE'], 'RMSE': m['RMSE'], 'R2': m['R2'],
                'AvgRelMAE': avg_rel, 'Time': elapsed
            })
            print(f"    {name:20s} | {elapsed:5.1f}s | MAE={m['MAE']:.1f} | RelMAE={avg_rel:.1f}% | R2={m['R2']:.3f}")

In [ ]:
def create_lstm_objective(nf_train, nf_val, hist_exog, horizon):
    def objective(trial):
        params = {
            'h': horizon,
            'input_size': INPUT_SIZE,
            'encoder_hidden_size': trial.suggest_int('hidden_size', 32, 256),
            'encoder_n_layers': trial.suggest_int('n_layers', 1, 3),
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
            'encoder_dropout': trial.suggest_float('dropout', 0.0, 0.5),
            'max_steps': 200,
            'early_stop_patience_steps': -1,
            'batch_size': BATCH_SIZE,
            'scaler_type': 'robust',
            'random_seed': 42,
            'accelerator': ACCELERATOR,
            'devices': DEVICES,
            'hist_exog_list': hist_exog,
            'loss': MAE(),
        }
        
        model = LSTM(**params)
        nf = NeuralForecast(models=[model], freq='h')
        
        nf_all = pd.concat([nf_train, nf_val]).sort_values(['unique_id', 'ds']).reset_index(drop=True)
        cv_results = nf.cross_validation(df=nf_all, n_windows=1, step_size=horizon)
        
        pred_col = [c for c in cv_results.columns if c not in ['unique_id', 'ds', 'cutoff', 'y']][0]
        y_true = cv_results['y'].values
        y_pred = np.clip(cv_results[pred_col].values, 0, None)
        groups = cv_results['unique_id'].values
        
        return calc_avg_rel_mae(y_true, y_pred, groups)
    return objective

if RUN_OPTUNA and HAS_NF:
    print("\n" + "=" * 60)
    print("OPTUNA LSTM OPTIMIZATION (optimizing AvgRelMAE)")
    print("=" * 60)
    
    nf_tr = prepared_data['Night0']['train']
    
    split_idx = int(len(nf_tr) * 0.82)
    nf_tr_sub = nf_tr.iloc[:split_idx]
    nf_va_sub = nf_tr.iloc[split_idx:]
    
    horizon_opt = min(24, nf_va_sub.groupby('unique_id').size().min())
    
    print(f"  Optimizing LSTM (horizon={horizon_opt})...")
    study = optuna.create_study(direction='minimize')
    
    try:
        study.optimize(
            create_lstm_objective(nf_tr_sub, nf_va_sub, ALL_FEATURES, horizon_opt),
            n_trials=min(10, OPTUNA_TRIALS),
            timeout=OPTUNA_TIMEOUT,
            show_progress_bar=True
        )
        
        print(f"    Best AvgRelMAE: {study.best_value:.2f}%")
        print(f"    Best params: {study.best_params}")
        best_models_per_dataset.setdefault('Night0', {})['LSTM_Optuna'] = study.best_params
    except Exception as e:
        print(f"    ERROR: {e}")

In [ ]:
lstm_params = best_models_per_dataset.get('Night0', {}).get('LSTM_Optuna')

if RUN_OPTUNA and HAS_NF and lstm_params:
    for ds_name in ['Full24h', 'Night0']:
        nf_train = prepared_data[ds_name]['train']
        nf_test = prepared_data[ds_name]['test']
        nf_all = pd.concat([nf_train, nf_test]).sort_values(['unique_id', 'ds']).reset_index(drop=True)
        
        horizon = min(72, nf_test.groupby('unique_id').size().min())
        
        print(f"\n  Evaluating LSTM_Optuna on {ds_name} (horizon={horizon})...")
        
        t0 = time.time()
        model = LSTM(
            h=horizon,
            input_size=INPUT_SIZE,
            encoder_hidden_size=lstm_params.get('hidden_size', 64),
            encoder_n_layers=lstm_params.get('n_layers', 2),
            learning_rate=lstm_params.get('learning_rate', LEARNING_RATE),
            encoder_dropout=lstm_params.get('dropout', 0.1),
            max_steps=MAX_STEPS,
            early_stop_patience_steps=-1,
            batch_size=BATCH_SIZE,
            scaler_type='robust',
            random_seed=42,
            accelerator=ACCELERATOR,
            devices=DEVICES,
            hist_exog_list=ALL_FEATURES,
            loss=MAE(),
        )
        
        nf = NeuralForecast(models=[model], freq='h')
        cv_results = nf.cross_validation(df=nf_all, n_windows=1, step_size=horizon)
        elapsed = time.time() - t0
        
        pred_col = [c for c in cv_results.columns if c not in ['unique_id', 'ds', 'cutoff', 'y']][0]
        y_true = cv_results['y'].values
        y_pred = np.clip(cv_results[pred_col].values, 0, None)
        
        m = calc_metrics(y_true, y_pred, y_true.max())
        eval_df = cv_results.copy()
        eval_df['beach'] = eval_df['unique_id']
        beach_df = eval_per_beach(eval_df, y_pred, 'beach')
        avg_rel = beach_df['RelMAE'].mean()
        
        all_results.append({
            'Model': 'LSTM_Optuna', 'Dataset': ds_name, 'Type': 'Optuna+NF',
            'MAE': m['MAE'], 'RMSE': m['RMSE'], 'R2': m['R2'],
            'AvgRelMAE': avg_rel, 'Time': elapsed
        })
        print(f"    {elapsed:.1f}s | MAE={m['MAE']:.1f} | RelMAE={avg_rel:.1f}% | R2={m['R2']:.3f}")

## Results

In [ ]:
results_df = pd.DataFrame(all_results)
beach_results_df = pd.concat(all_beach_results, ignore_index=True) if all_beach_results else pd.DataFrame()

save_dir = Path(SAVE_DIR)
save_dir.mkdir(parents=True, exist_ok=True)
results_df.to_csv(save_dir / 'results.csv', index=False)
if len(beach_results_df) > 0:
    beach_results_df.to_csv(save_dir / 'beach_results.csv', index=False)

print("\n" + "=" * 70)
print("RESULTS BY DATASET")
print("=" * 70)
for ds in datasets.keys():
    sub = results_df[results_df['Dataset'] == ds].sort_values('AvgRelMAE')
    if len(sub) == 0:
        continue
    print(f"\n{ds}:")
    print(sub[['Model', 'Type', 'MAE', 'R2', 'AvgRelMAE', 'Time']].to_string(index=False))

In [ ]:
pivot = results_df.pivot_table(index='Model', columns='Dataset', values='AvgRelMAE')
print("\nRelMAE (%) by Model x Dataset:")
print(pivot.round(1).to_string())

In [ ]:
if len(results_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    pivot = results_df.pivot_table(index='Model', columns='Dataset', values='AvgRelMAE')
    pivot = pivot.loc[pivot.mean(axis=1).sort_values().index]
    pivot.plot(kind='bar', ax=axes[0], width=0.8)
    axes[0].set_ylabel('Avg RelMAE (%)')
    axes[0].set_title('Model Performance (lower is better)')
    axes[0].legend(title='Dataset')
    axes[0].tick_params(axis='x', rotation=45)
    
    pivot_r2 = results_df.pivot_table(index='Model', columns='Dataset', values='R2')
    pivot_r2 = pivot_r2.loc[pivot_r2.mean(axis=1).sort_values(ascending=False).index]
    pivot_r2.plot(kind='bar', ax=axes[1], width=0.8)
    axes[1].set_ylabel('R²')
    axes[1].set_title('R² Score (higher is better)')
    axes[1].legend(title='Dataset')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(save_dir / 'comparison.png', dpi=150)
    plt.show()

In [ ]:
print("\n" + "=" * 70)
print("BEST MODEL PER DATASET")
print("=" * 70)

for ds in datasets.keys():
    sub = results_df[results_df['Dataset'] == ds].dropna(subset=['AvgRelMAE'])
    if len(sub) == 0:
        continue
    best = sub.loc[sub['AvgRelMAE'].idxmin()]
    print(f"\n{ds}: Best = {best['Model']} ({best['Type']})")
    print(f"  MAE: {best['MAE']:.2f}")
    print(f"  RelMAE: {best['AvgRelMAE']:.1f}%")
    print(f"  R²: {best['R2']:.3f}")
    print(f"  Time: {best['Time']:.1f}s")